# Anonimización Pulpocon 2023

## Datos y privacidad

En la era del Big Data, la información es uno de los bienes más preciados, y esto es así porque podemos sacar valor de ella gracias a la gran cantidad de información disponible y a las técnicas de análisis de datos y machine learning que se han desarrollado.

Este valor pasa por ofrecer más y mejores productos y servicios o por mejorar la eficiencia y eficacia de la organización, pero también por garantizar su seguridad y la de sus clientes.

La seguridad no solo hace ahorrar tiempo y dinero en solventar los incidentes que se puedan producir, sino que también aumentará la confianza y satisfacción con nuestra organización.

En este contexto, la privacidad de los dueños de la información disponible está en peligro. Se hace necesario controlar y proteger la información personal, todos aquellos datos que sobre una persona identificable por un atributo o la combinación de varios.

La UE estableció el GDPR para brindar protección sobre nuestros datos personales. Exige un consentimiento claro y específico para tratar datos personales, da más control a los ciudadanos sobre sus datos y obliga a las empresas a proteger esa información. Además, tiene carácter retroactivo, los que prácticamente imposibilita el procesamiento de datos históricos.

Por suerte, tenemos herramientas para proteger datos personales, tratando de mantener su valor, hasta el punto en el que es imposible, o casi, identificar los dueños originales de los datos, haciendo que no sean datos personales y, por tanto, ya no es necesario acogerse a la medidas específicas de la GDPR: **la anonimización**.

### Dataset de detección de fraude bancario

Para esta parte de anonimización vamos a utilizar un dataset bancario sobre los préstamos a sus clientes.

Dataset: https://www.kaggle.com/datasets/mishra5001/credit-card

De momento, no nos vamos a centrar demasiado en el significado real de estos datos, ya que la anonimización sería un paso previo a su procesamiento.

In [ ]:
import typing

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## Leemos el csv y vemos que pinta tiene

data = pd.read_csv('data/application_data_selected.csv')

data.info()

El dataset original tiene muchísimas columnas, así que he seleccionado unas pocas que se entienden bien para que el taller sea más ágil.

In [ ]:
## Descomentamos esto si utilizamos el dataset original

# selected_cols = [
#     'SK_ID_CURR',
#     'AMT_INCOME_TOTAL',
#     'AMT_CREDIT',
#     'DAYS_BIRTH',
#     'DAYS_EMPLOYED',
#     'TARGET',
#  ]

# data = data[selected_cols].copy()

In [ ]:
data.head()

Descripción de la columnas:

- SK_ID_CURR: id del préstamo.

- AMT_INCOME_TOTAL: ingresos del cliente.

- AMT_CREDIT: importe del préstamo.

- DAYS_BIRTH: edad del cliente en días en el momento de la solicitud.

- DAYS_EMPLOYED: cuántos días antes de la solicitud la persona empezó a trabajar.

- TARGET: cliente con dificultades de pago (1 - tuvo retrasos en el pago de más de X días en al menos una de las primeras Y cuotas del préstamo, 0 - todos los demás casos

Como vemos, contiene información que pueden ayudar a identificar a los clientes, además de información sensible como es la financiera.

Con un profiler, u otras herramientas de visualización, podemos tener una mejor idea de como son nuestros datos

In [ ]:
## Profiling del dataset, muy útil para hacer exploración de datos

# prof = ProfileReport(data)
# prof.to_file(output_file='reports/application_data_report.html')

In [ ]:
## Plots para ver la distribución de los datos

# sns.pairplot(data.drop('SK_ID_CURR', axis=1).sample(10000, axis=0))

Tenemos que decidir qué columnas anonimizamos y, realmente, todas serían susceptibles, puesto que son datos personales. Sin embargo:

- Los identificadores solo sirven para identificar préstamos, no van a aportar información si los anonimizamos. Ya nos encargaremos más tarde de ellos.

- TARGET es una variable binaria, dificil de anonimizar sin romper su significado, ya que tiene dos posibles valores. Además, parece ser la variable a predecir en el futuro porcesamiento de los datos, así que trataremos de no tocarla si no es necesario.

In [ ]:
sensitive_cols = [
    'AMT_INCOME_TOTAL',
    'AMT_CREDIT',
    'DAYS_BIRTH',
    'DAYS_EMPLOYED'
 ]

## Anonimización

Eliminar o reducir al mínimo el riesgo de identificación de las personas a partir de un conjunto de datos.

Se debe asumir que el adversario tiene la mayor cantidad posible de información externa sobre los usuarios, y evaluar así el riesgo de re-identificación.

Los dos principales mecanismos para anonimizar un dataset son:

- **Generalización**: se busca agregar los datos de distintos usuarios para evitar su re-identificación.

- **Randomización**: se busca añadir una distorsión a los datos, de modo que no se puedan relacionar con su valor original.

Es necesario tener en cuenta que estas transformaciones de los datos hacen que pierdan parte de su significado original, pero suelen ser necesarias para garantizar una cierta privacidad. Típicamente, cuánto más agresiva la anonimización, más información se perderá, haciendo que los datos pierdan utilidad para futuros usos, como sacar modelos estadísticos de ellos.

En estos contextos, es necesario jugar con la privacidad y la utilidad para alcanzar el equilibrio deseado entre ambas, por lo que precisamos de formas de medir ambas variables.

### K-anonymity

Implica ocultar la identidad de los individuos en un conjunto de datos al asegurarse de que cada fila sea indistinguible entre al menos "k" otros registros en términos de atributos compartidos, protegiendo así la privacidad mientras se mantiene la utilidad de los datos. Está muy orientada a la privacidad por generalización.

"K", por tanto, funciona como una métrica de privacidad: cuanto mayor sea "k" en nuestro dataset, mayor privacidad tendremos.

En la práctica, "k" se calcula como el número de veces que se repite en nuestro dataset la combinación de valores de una fila que menos se repite. Sería como hacer un group_by de todas las columnas y ver el grupo que menos registros contiene.

<img src="images/kanonymity.png" width="500"/>

In [ ]:
## Función para calcular k-anonimity

def k_anonymity(df: pd.DataFrame, return_sizes:bool=False) -> typing.Union[list, int]:
    group_sizes = list(df.value_counts())  # group_sizes = list(df.groupby(list(df.columns)).size())
    if return_sizes:
        return group_sizes
    
    return min(group_sizes)

In [ ]:
k_anonymity(data)

El "k" de nuestro dataset inicial es 1, lo que quiere decir que hay filas cuya combinación de valores es única. Pero era esperable, ya que sabíamos que hay identificadores únicos en la primera columna.

**Primer paso de la anonimización**: eliminar identificadores únicos

In [ ]:
data.drop('SK_ID_CURR', axis='columns', inplace=True)
k_anonymity(data)

Vemos que sigue sin cumplir k-anonimity, así que será necesario transformar estos datos para garantizar su privacidad.

### MSE (Error cuadrático medio)

El MSE es una de las métricas más utilizada para medir pérdida de utilidad. Cuanto mayor sea el error, más información perdemos.

$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

Es importante notar que este error saldrá elevado al cuadrado, para saber el error medio en la escala de nuestros datos será necesario hacer la raíz cuadrada. En la siguiente implementación bastará con pasarle *squared = False*.

In [ ]:
## Función que calcula el error entre dos datasets

def mse_by_col(df1: pd.DataFrame, df2:pd.DataFrame, squared:bool=True) -> pd.Series:
    if df1.columns.difference(df2.columns).size > 0 or df1.shape != df2.shape:
        raise ValueError('Dataframes must have same shape and columns')
    
    result = []
    cols = df1.columns
    
    for col in cols:
        result.append(mean_squared_error(df1[col], df2[col], squared=squared))
        
    return pd.Series(result, index=cols)

Para facilitar la evaluación de los datos anonimizados, a continuación dejo una función que saca tanto el K como el error con respecto a los datos originales

In [ ]:
## Función de evaluación

def evaluate_anonymization(df_original, df_anonym, squared_error:bool=True) -> None:
    group_sizes = k_anonymity(df_anonym, return_sizes=True)
    mse_by_columns = mse_by_col(df_original, df_anonym, squared=squared_error)
    k = min(group_sizes)
    
    print('Métrica K del dataset:', k)
    print('\nNúmero de grupos:', len(group_sizes))
    if k == 1:
        print('\nNúmero de registros únicos:', group_sizes.count(k))
    print('\n\nMSE por columnas:')
    print(mse_by_columns)
    print('\nMSE promedio:', mse_by_columns.mean())

### Mecanismos de generalización

La operaciones de generalización más comunes sobre las columnas de un dataset son clustering, redondeo, categorización y microagregaciones.

#### K-means

K-Means es un algoritmo de clustering no supervisado que agrupa un conjunto de datos en "k" grupos basados en sus características similares.

Funciona asignando puntos de datos al grupo cuyo centroide (punto medio) esté más cercano, y luego actualiza los centroides al calcular el promedio de los puntos asignados. Este proceso se repite iterativamente hasta que los centroides convergen y los grupos se vuelven estables.

<img src="images/kmeans.png" width="400"/>

In [ ]:
# Función para generalizar con k-means un dataset

def k_means(df:pd.DataFrame, n_clusters:int=8) -> pd.DataFrame:
    kmeans = KMeans(n_clusters=n_clusters, random_state=40)
    kmeans.fit(df)
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_

    return pd.DataFrame(centroids[labels], columns=df.columns)

##### Primera forma de generalizar: Kmeans de todo el dataset

In [ ]:
# Hacemos una copia del dataset original para modificar solo las columnas que queremos generalizar

data_kmeans = data.copy()
data_kmeans[sensitive_cols] = k_means(data[sensitive_cols], n_clusters=4)
data_kmeans

In [ ]:
# Vemos que tal fue la generalización con k-means

evaluate_anonymization(data, data_kmeans, squared_error=False)

Vaya... esto no era lo que esperaba. Hemos encontrado un registro que no conseguimos agrupar con ningún otro de los 307510, ni usando tan solo 4 grupos.

In [ ]:
# Buscamos el outlier

data_kmeans.value_counts()

In [ ]:
# Localizamos la fila

data_kmeans[data_kmeans['AMT_INCOME_TOTAL'] == 1.170000e+08]

Hemos encontrado un outlier bastante notable en los ingresos totales, así que lo vamos a eliminar y volver a ejecutar K-means, a ver si así no tenemos grupos tan pequeños y mejoramos la privacidad

In [ ]:
# Eliminamos el outlier

data.drop(12840, axis=0, inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
# Volvemos a aplicar k-means

data_kmeans = data.copy()
data_kmeans[sensitive_cols] = k_means(data[sensitive_cols], n_clusters=4)
data_kmeans

In [ ]:
# Volvemos a evaluar la generalización

evaluate_anonymization(data, data_kmeans, squared_error=False)

Podemos ver como tenemos un K de 1603, lo que implica un riesgo de re-identificación de 0.0624%, por lo que la privacidad es bastante alta. Sin embargo, al hacer tan solo 4 grupos de valores en todo el dataset, tenemos mucha pérdida de utilidad. Tal vez haciendo más grupos conseguiríamos mejorar la utilidad sin sacrificar demasiada privacidad.

##### Segunda forma de generalizar: Kmeans por columnas

In [ ]:
# Función que generaliza on k-means un dataset columna a columna

def k_means_by_col(df:pd.DataFrame, n_clusters:int=8) -> pd.DataFrame:
    new_df = df.copy()
    for col in df.columns:
        new_df[[col]] = k_means(df[[col]], n_clusters)
        
    return new_df

Al generalizar por columnas perdemos menos utilidad, pero tenemos muchas posibles combinaciones de valores cuando el número de columnas es muy grande, por lo que es fácil que haya combinaciones únicas

In [ ]:
# Aplicamos k-means por columnas

data_kmeans_bycol = data.copy()
data_kmeans_bycol[sensitive_cols] = k_means_by_col(data[sensitive_cols], n_clusters=4)
data_kmeans_bycol

In [ ]:
# Evaluamos la generalización por columnas

evaluate_anonymization(data, data_kmeans_bycol, squared_error=False)

Parece que en este caso tenemos, más o menos, la mitad de pérdida de información que antes, pero no hemos conseguido cumplir k-anonimity. Hay 9 combinaciones de valores que son únicas. Tal vez haciendo menos grupos en alguna columna lo conseguiríamos.

#### Microagregaciones

Es una manera de garantizar k-anonymity que consiste en ir haciendo grupos de "k" registros similares de forma iterativa, hasta que todos los registros estén en un grupo de, al menos, ese tamaño.

El algoritmo recibe un conjunto de datos y un *k*. Comienza calculando la fila promedio de todo el dataset. Después, coge la fila que más se aleje de la media (fila *r*) para que sea el centro del primer grupo. A continuación, coge la fila más alejada de *r* (fila *s*) para ser el centro del otro grupo. Al rededor de ambas se forman dos grupos de tamaño *k*, con las *k-1* filas más cercanas a *r* y a *s*. Esos registros se descartan del conjunto de entrada y se vuelve a empezar, hasta que queden menos de 2*k* filas, que forman el último grupo.

<img src="images/microaggregations.png" width="500"/>

In [ ]:
# Función para generalizar con microagregaciones

def euclidean_distance(v1, v2):
    if type(v1) == pd.DataFrame or type(v2) == pd.DataFrame:
        return np.sqrt((v2 - v1).pow(2).sum(axis=1))
    else:
        return np.sqrt(np.sum((v2 - v1)**2))

def microaggregation(df, k):
    input_df = df.copy()
    output_df = df.copy()
    while len(input_df) >= 3*k:
        # (a) Compute the average record x˜ of all records in R. The average record is computed attribute-wise.
        # (b) Consider the most distant record xr to the average record x˜ using an appropriate distance.
        # (c) Find the most distant record xs from the record xr considered in the previous step.
        # (d) Form two clusters around xr and xs, respectively. One cluster contains xr and the k −1 records closest to xr. The other cluster contains xs and the k −1 records closest to xs.
        # (e) Take as a new dataset R the previous dataset R minus the clusters formed around xr and xs in the last instance of Step 1d
        
        mean_row = input_df.mean(axis=0)
        
        # r: Cluster más lejano a la media actual
        r_index = euclidean_distance(mean_row, input_df).idxmax()
        r_row = input_df.loc[r_index]
        r_distances = euclidean_distance(r_row, input_df)
        r_cluster_indexes = r_distances.nsmallest(k).index
        r_cluster = input_df.loc[r_cluster_indexes]
        r_cluster_mean = r_cluster.mean(axis=0)
        
        # s: Cluster más lejano a r
        s_index = r_distances.idxmax()
        s_row = input_df.loc[s_index]
        s_distances = euclidean_distance(s_row, input_df)
        s_cluster_indexes = s_distances.nsmallest(k).index
        s_cluster = input_df.loc[s_cluster_indexes]
        s_cluster_mean = s_cluster.mean(axis=0)
        
        # sustituimos en la salida los clusters por su media, y los eliminamos del conjunto de entrada
        output_df.loc[r_cluster.index] = r_cluster_mean.values
        output_df.loc[s_cluster.index] = s_cluster_mean.values
        
        input_df.drop(r_cluster.index, axis=0, inplace=True)
        input_df.drop(s_cluster.index, axis=0, inplace=True)
    
    if 3*k-1 > len(input_df) >= 2*k:
        # (a) compute the average record x˜ of the remaining records in R
        # (b) find the most distant record xr from x˜
        # (c) form a cluster containing xr and the k − 1 records closest to xr
        # (d) form another cluster containing the rest of records.
        
        mean_row = input_df.mean(axis=0)
        
        # r: Cluster más lejano a la media actual
        r_index = euclidean_distance(mean_row, input_df).idxmax()
        r_row = input_df.loc[r_index]
        r_distances = euclidean_distance(r_row, input_df)
        r_cluster_indexes = r_distances.nsmallest(k).index
        r_cluster = input_df.loc[r_cluster_indexes]
        r_cluster_mean = r_cluster.mean(axis=0)
        
        output_df.loc[r_cluster.index] = r_cluster_mean.values
        
        input_df.drop(r_cluster.index, axis=0, inplace=True)
        
        # s: Cluster con los registros restantes
        s_cluster = input_df
        s_cluster_mean = s_cluster.mean(axis=0)
        
        output_df.loc[s_cluster.index] = s_cluster_mean.values
        
        input_df.drop(s_cluster.index, axis=0, inplace=True)
        
    else:
        # form a new cluster with the remaining records
        
        # Cluster restante
        s_cluster = input_df
        s_cluster_mean = s_cluster.mean(axis=0)
        
        output_df.loc[s_cluster.index] = s_cluster_mean.values
        
        input_df.drop(s_cluster.index, axis=0, inplace=True)
        
    return output_df

In [ ]:
# Aplicamos microagregaciones

data_microagg = data.copy()
data_microagg[sensitive_cols] = microaggregation(data[sensitive_cols], k=500)
data_microagg

In [ ]:
# Evaluamos la generalización con microagregaciones

evaluate_anonymization(data, data_microagg, squared_error=False)

Debemos tener en cuenta que TARGET no la agrupamos, motivo por el que no obtenemos un "k" de 500. Si incluyesemos esa columna en las microagregaciones, sí lo conseguiríamos, pero perderíamos utilidad de la variable objetivo.

**Ejercicio**: Mejorar los resultados privacidad/utilidad de las técnicas vistas. Se puede modificar *k*, el número de clusters y las columnas que se utilizan para agregar. También se puede modificar la parte que querais de los algortimos para hacerlos más rápidos o para reducir la pérdida de información.